In [ ]:
from tensorflow import keras
import kerastuner

# 使用keras tuner来寻找超参数

# 该函数返回已编译的模型
def build_model(hp):
    inputs = keras.Input(shape=(20,),
                         name='in',
                         dtype='float32')
    x = keras.layers.Dense(units=hp.Int('units', min_value=32, max_value=128, step=16),
                           activation='relu', name='dense_1')(inputs)
    outputs = keras.layers.Dense(10, activation='softmax')(x)
    model = keras.models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate',
                                                                          values=[0.1, 0.02])),
                  loss=keras.losses.sparse_categorical_crossentropy,
                  metrics=[keras.metrics.CategoricalAccuracy(name='ACC')])
    return model


# 实例化一个指定优化目标和其他搜索参数的tuner对象
tuner = kerastuner.tuners.Hyperband(build_model,
                                    objective='val_loss',
                                    max_epochs=100,
                                    max_trials=200,
                                    executions_per_trial=2,
                                    directory='my_dir')

# 从search开始搜，使用方法类似fit
import numpy as np
dataset = np.array([[]])
val_dataset = dataset
tuner.search(dataset, validation_data=val_dataset)
models = tuner.get_best_models(num_models=2)
print(tuner.results_summary())